In [1]:
import os
from configparser import ConfigParser

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [83]:
 spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

#spark = SparkSession.builder.config('spark.jars.packages','org.apache.hadoop.fs.s3a.S3AFileSystem').appName("os").getOrCreate()
#spark.conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

In [37]:
#spark
df_log = spark.read.json("/home/workspace/data/log-data.zip")
#!pwd

In [36]:
df_song.count()

2989

In [89]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import  IntegerType,StringType,DecimalType,LongType,DoubleType

song_schema = StructType([
    StructField('artist_id',StringType()),  
    StructField('artist_latitude',DoubleType()),
    StructField('artist_location',StringType()),
    StructField('artist_longitude',DoubleType()),
    StructField('artist_name',StringType()),
    StructField('duration',DoubleType()),
    StructField('num_songs' ,IntegerType()),
    StructField('song_id',StringType()),
    StructField('title',StringType()),
    StructField('year',IntegerType())   
])
log_schema = StructType([
    StructField('artist',StringType()),
    StructField('auth',StringType()),
    StructField('firstName',StringType()),
    StructField('gender',StringType()),
    StructField('itemInSession',IntegerType()),
    StructField('lastName',StringType()),
    StructField('length',DoubleType()),
    StructField('level',StringType()),
    StructField('location',StringType()),
    StructField('method',StringType()),
    StructField('page',StringType()),
    StructField('registration',StringType()),
    StructField('sessionId',IntegerType()),    
    StructField('song',StringType()),
    StructField('status',IntegerType()),
    StructField('ts',LongType()),
    StructField('userAgent',StringType()),
    StructField('userId',StringType())
    
])
df_song = spark.read.json("/home/workspace/data/song_data/*/*/*/*.json",schema=song_schema)
df_log = spark.read.json("/home/workspace/data/*.json",schema=log_schema)

#df_song = spark.read.json("s3a://udacity-dend/song_data/*/*/*/*.json",schema=song_schema)
#df_log = spark.read.json("s3a://udacity-dend/log_data/*/*/*.json",schema=log_schema)

In [90]:
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import to_timestamp
dim_user_columns = ['userid','firstname','lastName','gender','level']
dim_song_columns = ['song_id','artist_id','duration','title','year']
dim_artist_columns = ['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']

dim_user = df_log.selectExpr(*dim_user_columns).dropDuplicates().filter(col('userid').isNotNull()) 
dim_song = df_song.selectExpr(*dim_song_columns).dropDuplicates() 
dim_artist =  df_song.selectExpr(*dim_artist_columns).dropDuplicates()

df_log.selectExpr("cast(ts/1000 as timestamp) as time").alias('time').createOrReplaceTempView("time_table")
dim_time = spark.sql("select DISTINCT time, hour(time) as hour, day(time) as day,weekofyear(time) as week,\
          month(time) as month,year(time) as year,date_format(time, 'EEEE') as weekday  FROM time_table")


#####################################
dim_user.show(n=5)
dim_song.show(n=5)
dim_time.show(n=5)
dim_artist.show(n=5)

+------+---------+--------+------+-----+
|userid|firstname|lastName|gender|level|
+------+---------+--------+------+-----+
|    26|     Ryan|   Smith|     M| free|
|     7|   Adelyn|  Jordan|     F| free|
|    71|   Ayleen|    Wise|     F| free|
|    81|   Sienna|   Colon|     F| free|
|    87|   Dustin|     Lee|     M| free|
+------+---------+--------+------+-----+
only showing top 5 rows

+------------------+------------------+---------+--------------------+----+
|           song_id|         artist_id| duration|               title|year|
+------------------+------------------+---------+--------------------+----+
|SOHUOAP12A8AE488E9|ARD842G1187B997376|491.12771|            Floating|1987|
|SOKEJEJ12A8C13E0D0|ARC43071187B990240|245.21098|The Urgency (LP V...|   0|
|SONHOTT12A8C13493C|AR7G5I41187FB4CE6C|233.40363|     Something Girls|1982|
|SOHKNRJ12A6701D1F8|AR10USD1187B99F3F1|189.57016|        Drop of Rain|   0|
|SOOLYAZ12A6701F4A6|AREBBGV1187FB523D2|173.66159|Laws Patrolling (...|   0

In [91]:
from pyspark.sql.functions import monotonically_increasing_id 
from pyspark.sql.window import Window
from pyspark.sql import functions as F 
columns_l = ['start_time, user_id, level, song_id, artist_id, session_id, location, user_agent']
#df_log.selectExpr("cast( ts/1000 as timestamp)").show(n=5)
#df_songplay = df_log.where(col('page') == 'NextSong').join(dim_song,df_log.song  == dim_song.title) \
#.selectExpr(*columns_l).show(5)
#df_songplay.limit(5).select( F.monotonically_increasing_id()).show()
df_log.where(col('page') == 'NextSong')\
.join(dim_song,df_log.song  == dim_song.title)\
.selectExpr('monotonically_increasing_id() as songplay_id','userId as user_id','level','song_id','artist_id','sessionId as session_id','location','userAgent as user_agent') \
.collect() 

[Row(songplay_id=0, user_id='15', level='paid', song_id='SOZCTXZ12AB0182364', artist_id='AR5KOSW1187FB35FF4', session_id=818, location='Chicago-Naperville-Elgin, IL-IN-WI', user_agent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"'),
 Row(songplay_id=1, user_id='10', level='free', song_id='SOGDBUF12A8C140FAA', artist_id='AR558FS1187FB45658', session_id=484, location='Washington-Arlington-Alexandria, DC-VA-MD-WV', user_agent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4"'),
 Row(songplay_id=2, user_id='24', level='paid', song_id='SOGDBUF12A8C140FAA', artist_id='AR558FS1187FB45658', session_id=672, location='Lake Havasu City-Kingman, AZ', user_agent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"'),
 Row(songplay_id=3, user_id='80', level='paid', song_id='SOGDB

In [88]:
df_songplay.columns

['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'method',
 'page',
 'registration',
 'sessionId',
 'song',
 'status',
 'ts',
 'userAgent',
 'userId',
 'song_id',
 'artist_id',
 'duration',
 'title',
 'year']